<h1>Separating JSON Data into Columns (Movies)</h1>

In [91]:
import csv, json, gensim, datetime, time
from gensim import corpora

rowslist = []
genreslist = []
#productioncompanieslist = []
#productioncountrieslist = []

def readJSON(row, rownumber, list, keyword):
    jsonstring = row[rownumber]
    jsonstringa = jsonstring.replace("\'", "\"")
    jsonstringb = jsonstringa.replace("None", "null")
    jsonobj = json.loads(jsonstringb)

    for jsonelement in jsonobj:
        keywordelement = jsonelement[keyword]       
        list.append(keywordelement.lower()) 
    
    return list
    
#read file
with open('tmdb_10000_movies.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:        
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else: #if line_count != 5:
                eachnewrow = []
                genres = []
                productioncompanies = []
                productioncountries = []

                eachnewrow.append(row[0]) #budget

                #handle json genres
                genres = readJSON(row, 1, genres, 'name')

                eachnewrow.append(genres) #genres
                genreslist.append(genres)

                #row[2] homepage removed
                eachnewrow.append(row[3]) #id

                #row[4] original_language removed
                #row[5] original_title removed  
                #row[6] overview removed                    
                eachnewrow.append(row[7]) #popularity                  

                #handle json productioncompanies
                productioncompanies = readJSON(row, 8, productioncompanies, 'name')

                eachnewrow.append(len(productioncompanies)) #productioncompanies
                #productioncompanieslist.append(productioncompanies)                  

                #handle json productioncountries
                productioncountries = readJSON(row, 9, productioncountries, 'name')

                eachnewrow.append(len(productioncountries)) #productioncountries
                #productioncountrieslist.append(productioncountries) 

                weekday = ""
                day = ""
                month = ""
                year = ""                 

                #handle releasedate 
                if row[10] != "":
                    d = datetime.datetime.strptime(row[10] ,'%Y-%m-%d')

                    weekdayword = d.strftime('%a')
                    weekday = time.strptime(weekdayword, '%a').tm_wday + 1

                    day = d.strftime('%d')

                    monthword = d.strftime('%b')
                    month = time.strptime(monthword,'%b').tm_mon

                    year = d.strftime('%Y')

                eachnewrow.append(weekday) #weekday
                eachnewrow.append(day) #day 
                eachnewrow.append(month) #month
                eachnewrow.append(year) #year

                eachnewrow.append(row[11]) #revenue
                eachnewrow.append(row[12]) #runtime
                #row[13] spoken_languages removed  
                #row[14] status removed 
                #row[15] tagline removed   
                eachnewrow.append(row[16]) #title
                eachnewrow.append(row[17]) #vote_average
                eachnewrow.append(row[18]) #vote_count                  

                WR = (float(row[18]) / (float(row[18])+25000)) * float(row[17]) + (25000 / (float(row[18]) + 25000)) * 7.0            

                eachnewrow.append(WR) #weighted_rating

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception')    
            line_count += 1          
    print(f'Processed {line_count} lines.')   
                  
genresdictionary = corpora.Dictionary(genreslist)
token_to_id_genres = genresdictionary.token2id                

#productioncompaniesdictionary = corpora.Dictionary(productioncompanieslist)
#token_to_id_productioncompanies = productioncompaniesdictionary.token2id
                  
#productioncountriesdictionary = corpora.Dictionary(productioncountrieslist)
#token_to_id_productioncountries = productioncountriesdictionary.token2id   

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros
                  
#write to file
with open('processed_tmdb_10000_movies.csv', mode='w', newline='', encoding='utf-8') as movies_file: #change accordingly
    writer2 = csv.writer(movies_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer2.writerow(['budget'] + list(genresdictionary.values()) + ['id', 'popularity', 'production_companies', 'production_countries'] 
        #+ list(productioncompaniesdictionary.values()) + list(productioncountriesdictionary.values()) 
        + ['weekday', 'day', 'month', 'year', 'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'weighted_rating'])              
    
    for eachrow in rowslist:
        listofzerosgenres = [0] * len(token_to_id_genres)
        #listofzerosproductioncompanies = [0] * len(token_to_id_productioncompanies)
        #listofzerosproductioncountries = [0] * len(token_to_id_productioncountries)                   
                  
        vec_genres = genresdictionary.doc2bow(eachrow[1])
        #vec_productioncompanies = productioncompaniesdictionary.doc2bow(eachrow[4])
        #vec_productioncountries = productioncountriesdictionary.doc2bow(eachrow[5])                   

        listofzerosgenres = assigningValues(vec_genres, listofzerosgenres)
        #listofzerosproductioncompanies = assigningValues(vec_productioncompanies, listofzerosproductioncompanies)
        #listofzerosproductioncountries = assigningValues(vec_productioncountries, listofzerosproductioncountries)

        writer2.writerow(eachrow[0:1] + listofzerosgenres + eachrow[2:6] 
            #+ listofzerosproductioncompanies + listofzerosproductioncountries 
            + eachrow[6:16])
                  
    print('\nOutput Success!')                 

Column names are budget, genres, homepage, id, original_language, original_title, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count
Row: 6 has Exception
Row: 54 has Exception
Row: 76 has Exception
Row: 112 has Exception
Row: 120 has Exception
Row: 154 has Exception
Row: 200 has Exception
Row: 229 has Exception
Row: 274 has Exception
Row: 321 has Exception
Row: 388 has Exception
Row: 407 has Exception
Row: 432 has Exception
Row: 454 has Exception
Row: 463 has Exception
Row: 476 has Exception
Row: 573 has Exception
Row: 667 has Exception
Row: 675 has Exception
Row: 758 has Exception
Row: 840 has Exception
Row: 856 has Exception
Row: 859 has Exception
Row: 860 has Exception
Row: 892 has Exception
Row: 896 has Exception
Row: 897 has Exception
Row: 908 has Exception
Row: 1075 has Exception
Row: 1112 has Exception
Row: 1168 has Exception
Row: 1175 has Exception
Row: 1193 has Exc

<h1>Separating JSON Data into Columns (Credits)</h1>

In [94]:
import csv, json, gensim
from gensim import corpora

rowslist = []
departmentslist = []
jobtitleslist = []

#read file
with open('tmdb_10000_credits.csv', encoding='utf-8') as csv_file: #change accordingly
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        try:
            if line_count == 0:
                print(f'Column names are {", ".join(row)}')
                line_count += 1
            else: #if line_count != 5:
                eachnewrow = []
                departments = []
                jobtitles = []

                malecastsnum = 0
                femalecastsnum = 0
                unknowngendercastsnum = 0    
                totalcasts = 0

                malecrewsnum = 0
                femalecrewsnum = 0
                unknowngendercrewsnum = 0
                totalcrews = 0

                eachnewrow.append(row[0]) #id

                #handle json casts
                jsonstring = row[1]
                jsonstringa = jsonstring.replace("\'", "\"")
                jsonstringb = jsonstringa.replace("None", "null")
                jsonobj = json.loads(jsonstringb)

                for jsonelement in jsonobj:
                    gender = jsonelement['gender']       
                    if gender == 0:
                      unknowngendercastsnum += 1
                    elif gender == 1:
                      femalecastsnum += 1
                    elif gender == 2:
                      malecastsnum += 1

                    totalcasts += 1

                eachnewrow.append(malecastsnum) #malecastsnum
                eachnewrow.append(femalecastsnum) #femalecastsnum                  
                eachnewrow.append(unknowngendercastsnum) #unknowngendercastsnum 
                eachnewrow.append(totalcasts) #totalcasts                   

                #handle json crews              
                jsonstring2 = row[2]
                jsonstring2a = jsonstring2.replace("\'", "\"")
                jsonstring2b = jsonstring2a.replace("None", "null")
                jsonobj2 = json.loads(jsonstring2b)

                for jsonelement in jsonobj2:
                    gender = jsonelement['gender']       
                    if gender == 0:
                      unknowngendercrewsnum += 1
                    elif gender == 1:
                      femalecrewsnum += 1
                    elif gender == 2:
                      malecrewsnum += 1  

                    totalcrews += 1

                    department = jsonelement['department']
                    departments.append(department.lower())         

                    jobtitle = jsonelement['job']
                    jobtitles.append(jobtitle.lower())                    

                eachnewrow.append(malecrewsnum) #malecastsnum
                eachnewrow.append(femalecrewsnum) #femalecastsnum                  
                eachnewrow.append(unknowngendercrewsnum) #unknowngendercastsnum                   
                eachnewrow.append(totalcrews) #totalcrews  

                eachnewrow.append(departments) #departments
                departmentslist.append(departments)  

                eachnewrow.append(jobtitles) #jobtitles
                jobtitleslist.append(jobtitles)

                rowslist.append(eachnewrow)

                line_count += 1
        except Exception as e:
            print(f'Row: {line_count} has Exception')    
            line_count += 1   

    print(f'Processed {line_count} lines.')
                  
departmentsdictionary = corpora.Dictionary(departmentslist)
token_to_id_departments = departmentsdictionary.token2id
#print(f'Dictionary for Departments: {token_to_id_departments}') 
#print(f'\nNo. of Departments: {len(token_to_id_departments)}')
                  
jobtitlesdictionary = corpora.Dictionary(jobtitleslist)
token_to_id_jobtitles = jobtitlesdictionary.token2id
#print(f'Dictionary for Job Titles: {token_to_id_jobtitles}')        
#print(f'No. of Job Titles: {len(token_to_id_jobtitles)}')                   

def assigningValues(vector, listofzeros):
    for counter, value in enumerate(vector): 
        index = 0       
        for value2 in listofzeros:                      
            if index == value[0]:
                listofzeros[index] = value[1]    
            index += 1
                  
    return listofzeros                  
                  
#write to file
with open('processed_tmdb_10000_credits.csv', mode='w', newline='', encoding='utf-8') as credits_file: #change accordingly
    writer = csv.writer(credits_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    writer.writerow(['id', 'malecastsnum', 'femalecastsnum', 'unknowngendercastsnum', 'totalcasts', 'malecrewsnum', 'femalecrewsnum', 'unknowngendercrewsnum', 'totalcrews'] 
        + list(departmentsdictionary.values()) + list(jobtitlesdictionary.values()))              
    
    for eachrow in rowslist:
        listofzerosdepartments = [0] * len(token_to_id_departments)
        listofzerosjobtitles = [0] * len(token_to_id_jobtitles)                         
                  
        vec_departments = departmentsdictionary.doc2bow(eachrow[9])
        vec_jobtitles = jobtitlesdictionary.doc2bow(eachrow[10])   

        listofzerosdepartments = assigningValues(vec_departments, listofzerosdepartments)
        listofzerosjobtitles = assigningValues(vec_jobtitles, listofzerosjobtitles)

        writer.writerow(eachrow[0:9] + listofzerosdepartments + listofzerosjobtitles)

    print('\nOutput Success!')
                  

Column names are id, cast, crew, , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , , ,

Expecting ',' delimiter: line 1 column 6052 (char 6051)
Row: 521 has Exception
Expecting ',' delimiter: line 1 column 2039 (char 2038)
Row: 522 has Exception
Expecting ',' delimiter: line 1 column 2266 (char 2265)
Row: 523 has Exception
Expecting ',' delimiter: line 1 column 3392 (char 3391)
Row: 524 has Exception
Expecting ',' delimiter: line 1 column 3285 (char 3284)
Row: 525 has Exception
Expecting ',' delimiter: line 1 column 1654 (char 1653)
Row: 526 has Exception
Expecting ',' delimiter: line 1 column 3152 (char 3151)
Row: 528 has Exception
Expecting ',' delimiter: line 1 column 1932 (char 1931)
Row: 530 has Exception
Expecting ',' delimiter: line 1 column 3814 (char 3813)
Row: 531 has Exception
Expecting ',' delimiter: line 1 column 646 (char 645)
Row: 535 has Exception
Expecting ',' delimiter: line 1 column 1452 (char 1451)
Row: 540 has Exception
Expecting ',' delimiter: line 1 column 1116 (char 1115)
Row: 542 has Exception
Expecting ',' delimiter: line 1 column 2960 (char 2959

Expecting ',' delimiter: line 1 column 2130 (char 2129)
Row: 1229 has Exception
Expecting value: line 1 column 1 (char 0)
Row: 1232 has Exception
Expecting ',' delimiter: line 1 column 739 (char 738)
Row: 1236 has Exception
Expecting ',' delimiter: line 1 column 121 (char 120)
Row: 1237 has Exception
Expecting ',' delimiter: line 1 column 415 (char 414)
Row: 1238 has Exception
Expecting ',' delimiter: line 1 column 1215 (char 1214)
Row: 1242 has Exception
Expecting ',' delimiter: line 1 column 3216 (char 3215)
Row: 1244 has Exception
Expecting ',' delimiter: line 1 column 2134 (char 2133)
Row: 1246 has Exception
Expecting ',' delimiter: line 1 column 44 (char 43)
Row: 1250 has Exception
Expecting ',' delimiter: line 1 column 242 (char 241)
Row: 1253 has Exception
Expecting ',' delimiter: line 1 column 2021 (char 2020)
Row: 1254 has Exception
Expecting ',' delimiter: line 1 column 648 (char 647)
Row: 1257 has Exception
Expecting ',' delimiter: line 1 column 1017 (char 1016)
Row: 1262 ha

Expecting ',' delimiter: line 1 column 3187 (char 3186)
Row: 1769 has Exception
Expecting value: line 1 column 1 (char 0)
Row: 1772 has Exception
Expecting ',' delimiter: line 1 column 2623 (char 2622)
Row: 1774 has Exception
Expecting ',' delimiter: line 1 column 437 (char 436)
Row: 1779 has Exception
Expecting ',' delimiter: line 1 column 14410 (char 14409)
Row: 1780 has Exception
Expecting ',' delimiter: line 1 column 483 (char 482)
Row: 1783 has Exception
Expecting ',' delimiter: line 1 column 2830 (char 2829)
Row: 1784 has Exception
Expecting ',' delimiter: line 1 column 1701 (char 1700)
Row: 1785 has Exception
Expecting ',' delimiter: line 1 column 16823 (char 16822)
Row: 1786 has Exception
Extra data: line 1 column 8 (char 7)
Row: 1787 has Exception
Expecting value: line 1 column 1 (char 0)
Row: 1788 has Exception
Expecting ',' delimiter: line 1 column 47 (char 46)
Row: 1789 has Exception
Expecting value: line 1 column 1 (char 0)
Row: 1790 has Exception
Expecting ',' delimiter: 

Expecting ',' delimiter: line 1 column 1024 (char 1023)
Row: 2275 has Exception
Expecting ',' delimiter: line 1 column 1010 (char 1009)
Row: 2276 has Exception
Expecting ',' delimiter: line 1 column 2987 (char 2986)
Row: 2277 has Exception
Expecting ',' delimiter: line 1 column 2904 (char 2903)
Row: 2278 has Exception
Expecting ',' delimiter: line 1 column 332 (char 331)
Row: 2281 has Exception
Expecting ',' delimiter: line 1 column 423 (char 422)
Row: 2282 has Exception
Expecting ',' delimiter: line 1 column 2945 (char 2944)
Row: 2283 has Exception
Expecting ',' delimiter: line 1 column 3874 (char 3873)
Row: 2289 has Exception
Expecting ',' delimiter: line 1 column 3806 (char 3805)
Row: 2291 has Exception
Expecting ',' delimiter: line 1 column 1793 (char 1792)
Row: 2294 has Exception
Expecting ',' delimiter: line 1 column 20656 (char 20655)
Row: 2295 has Exception
Expecting ',' delimiter: line 1 column 1896 (char 1895)
Row: 2297 has Exception
Expecting ',' delimiter: line 1 column 142

...

Expecting ',' delimiter: line 1 column 4559 (char 4558)
Row: 9747 has Exception
Expecting ',' delimiter: line 1 column 354 (char 353)
Row: 9749 has Exception
Expecting ',' delimiter: line 1 column 3223 (char 3222)
Row: 9755 has Exception
Expecting ',' delimiter: line 1 column 2070 (char 2069)
Row: 9756 has Exception
Expecting ',' delimiter: line 1 column 2658 (char 2657)
Row: 9757 has Exception
Expecting ',' delimiter: line 1 column 131 (char 130)
Row: 9759 has Exception
Expecting ',' delimiter: line 1 column 342 (char 341)
Row: 9762 has Exception
Expecting ',' delimiter: line 1 column 148 (char 147)
Row: 9764 has Exception
Expecting ',' delimiter: line 1 column 631 (char 630)
Row: 9765 has Exception
Expecting ',' delimiter: line 1 column 180 (char 179)
Row: 9766 has Exception
Expecting ',' delimiter: line 1 column 425 (char 424)
Row: 9767 has Exception
Expecting ',' delimiter: line 1 column 38 (char 37)
Row: 9768 has Exception
Expecting ',' delimiter: line 1 column 181 (char 180)
Row:

Expecting ',' delimiter: line 1 column 2904 (char 2903)
Row: 10232 has Exception
Expecting ',' delimiter: line 1 column 872 (char 871)
Row: 10233 has Exception
Expecting ',' delimiter: line 1 column 3741 (char 3740)
Row: 10235 has Exception
Expecting ',' delimiter: line 1 column 737 (char 736)
Row: 10243 has Exception
Expecting ',' delimiter: line 1 column 853 (char 852)
Row: 10249 has Exception
Expecting ',' delimiter: line 1 column 2128 (char 2127)
Row: 10253 has Exception
Expecting ',' delimiter: line 1 column 1550 (char 1549)
Row: 10255 has Exception
Expecting ',' delimiter: line 1 column 586 (char 585)
Row: 10257 has Exception
Expecting ',' delimiter: line 1 column 5640 (char 5639)
Row: 10258 has Exception
Expecting ',' delimiter: line 1 column 2436 (char 2435)
Row: 10266 has Exception
Expecting ',' delimiter: line 1 column 2389 (char 2388)
Row: 10267 has Exception
Expecting ',' delimiter: line 1 column 1768 (char 1767)
Row: 10269 has Exception
Expecting ',' delimiter: line 1 colu

Expecting ',' delimiter: line 1 column 654 (char 653)
Row: 10791 has Exception
Expecting ',' delimiter: line 1 column 567 (char 566)
Row: 10793 has Exception
Expecting ',' delimiter: line 1 column 2602 (char 2601)
Row: 10794 has Exception
Expecting ',' delimiter: line 1 column 269 (char 268)
Row: 10795 has Exception
Expecting ',' delimiter: line 1 column 1534 (char 1533)
Row: 10799 has Exception
Expecting ',' delimiter: line 1 column 793 (char 792)
Row: 10803 has Exception
Expecting ',' delimiter: line 1 column 4409 (char 4408)
Row: 10804 has Exception
Expecting ',' delimiter: line 1 column 3350 (char 3349)
Row: 10806 has Exception
Expecting ',' delimiter: line 1 column 348 (char 347)
Row: 10808 has Exception
Expecting ',' delimiter: line 1 column 43 (char 42)
Row: 10811 has Exception
Expecting ',' delimiter: line 1 column 3612 (char 3611)
Row: 10812 has Exception
Expecting ',' delimiter: line 1 column 2152 (char 2151)
Row: 10815 has Exception
Expecting ',' delimiter: line 1 column 548

Expecting ',' delimiter: line 1 column 514 (char 513)
Row: 11394 has Exception
Expecting ',' delimiter: line 1 column 1301 (char 1300)
Row: 11395 has Exception
Expecting ',' delimiter: line 1 column 1248 (char 1247)
Row: 11396 has Exception
Expecting ',' delimiter: line 1 column 1303 (char 1302)
Row: 11397 has Exception
Expecting ',' delimiter: line 1 column 1415 (char 1414)
Row: 11405 has Exception
Expecting ',' delimiter: line 1 column 200 (char 199)
Row: 11407 has Exception
Expecting ',' delimiter: line 1 column 1062 (char 1061)
Row: 11408 has Exception
Expecting ',' delimiter: line 1 column 33 (char 32)
Row: 11410 has Exception
Expecting ',' delimiter: line 1 column 764 (char 763)
Row: 11412 has Exception
Expecting ',' delimiter: line 1 column 184 (char 183)
Row: 11413 has Exception
Expecting ',' delimiter: line 1 column 727 (char 726)
Row: 11416 has Exception
Expecting ',' delimiter: line 1 column 285 (char 284)
Row: 11417 has Exception
Expecting ',' delimiter: line 1 column 1235 


Output Success!


<h1>Merging Two Files</h1>

In [93]:
import pandas as pd

movies = pd.read_csv("processed_tmdb_10000_movies.csv") #change accordingly
credits = pd.read_csv("processed_tmdb_10000_credits.csv") #change accordingly
merged = pd.merge(movies, credits, left_on=['id'],
                   right_on=['id'], 
                   how='left')
merged.to_csv("processed_tmdb_data.csv", index=False)

print('\nOutput Success!')  


Output Success!


<h1>Append New Processed Data to Previous Merged File</h1>

In [36]:
import csv

with open('main_processed_tmdb_data.csv', mode='a', newline='', encoding='utf-8') as main_file:
    writer3 = csv.writer(main_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    with open('processed_tmdb_data.csv', mode='r', newline='') as new_file:
        csv_reader = csv.reader(new_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                writer3.writerow(row)
                line_count += 1
            
    print(f'Processed {line_count} lines.')
    
    print('\nOutput Success!')  

Processed 4804 lines.

Output Success!
